In [1]:
from keras.models import load_model
from collections import deque
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2

In [2]:
args = {
	"model": "mod/violence.h5",
    "label-bin": "mod/lb.pickle",
    "input": "./input/V_8.mp4",
    # "input": "./video/NonViolence/NV_1.mp4",
    # "input": "./input/street-fight.mp4",
    "output": "output/By-predict_video_(OLD MODEL).mp4",
    "size": 64
}

In [3]:
tf.compat.v1.disable_eager_execution()
# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model(args["model"])
lb = pickle.loads(open(args["label-bin"], "rb").read())


[INFO] loading model and label binarizer...


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=args["size"])

In [5]:
# initialize the video stream, pointer to output video file, and
# frame dimensions
vpath = args["input"]
if args["input"] == 'camera':
	vpath = 0
vs = cv2.VideoCapture(vpath)
writer = None
(W, H) = (None, None)

In [6]:
# ... (previous code)

# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()

    # if the frame was not grabbed, then we have reached the end of the stream
    if not grabbed:
        print("[INFO] End of video stream.")
        break

    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]

    # clone the output frame, then convert it from BGR to RGB ordering, resize the frame to a fixed 224x224, and then perform mean subtraction
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean

    # make predictions on the frame and then update the predictions queue
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)

    # perform prediction averaging over the current history of previous predictions
    results = np.array(Q).mean(axis=0)
    i = 1
    label = lb.classes_[i]

    prob = results[i] * 100
    text_color = (0, 255, 0)  # default: green

    if prob > 52:  # Violence prob
        text_color = (0, 0, 255)  # red
    else:
        label = 'Normal'

    text = "State : {:8} ({:3.2f}%)".format(label, prob)
    FONT = cv2.FONT_HERSHEY_SIMPLEX

    cv2.putText(output, text, (35, 50), FONT, 1.25, text_color, 3)

    # plot graph over background image
    output = cv2.rectangle(output, (35, 80), (35 + int(prob) * 5, 80 + 20), text_color, -1)

    # check if the video writer is None
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"H264")
        writer = cv2.VideoWriter(args["output"], fourcc, 30, (W, H), True)

    # write the output frame to disk
    writer.write(output)

    # show the output image
    cv2.imshow("Output", output)
    
    # introduce a slight delay and check for key press
    key = cv2.waitKey(1) & 0xFF

    # if the 'q' key was pressed or window is closed, break from the loop
    if key == ord("q") or cv2.getWindowProperty("Output", cv2.WND_PROP_VISIBLE) < 1:
        break

# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()
cv2.destroyAllWindows()  # Close all OpenCV windows


d:\Projects\RL\.venv\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[INFO] End of video stream.
[INFO] cleaning up...
